In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

filename = '/Users/mac/Desktop/BigData_ProjectData/SPARCS-2017.csv'
raw_data = pd.read_csv(filename)
raw_data.head()

/Users/mac/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
0,Hudson Valley,Westchester,5903001.0,1061.0,Montefiore Mount Vernon Hospital,30 to 49,NaN,M,White,Not Span/Hispanic,...,Minor,Medical,Medicare,Medicare,Self-Pay,NaN,N,N,114168.00,40464.04
1,Hudson Valley,Westchester,5903001.0,1061.0,Montefiore Mount Vernon Hospital,50 to 69,105,M,White,Spanish/Hispanic,...,Moderate,Medical,Medicaid,Medicaid,Self-Pay,NaN,N,N,58517.00,26046.50
2,Hudson Valley,Westchester,5903001.0,1061.0,Montefiore Mount Vernon Hospital,30 to 49,105,F,White,Unknown,...,Moderate,Medical,Department of Corrections,Self-Pay,NaN,NaN,N,N,39647.04,17397.59
3,Hudson Valley,Westchester,5903001.0,1061.0,Montefiore Mount Vernon Hospital,50 to 69,105,F,White,Not Span/Hispanic,...,Moderate,Medical,Medicaid,Medicaid,Self-Pay,NaN,N,N,36031.09,16118.84
4,Hudson Valley,Westchester,5903001.0,1061.0,Montefiore Mount Vernon Hospital,18 to 29,105,F,White,Spanish/Hispanic,...,Minor,Medical,Department of Corrections,Self-Pay,NaN,NaN,N,N,30534.00,11921.05


In [8]:
# Drop a few NA records
data = raw_data.dropna(subset = ['Hospital Service Area','Zip Code - 3 digits',
                             'APR Risk of Mortality'])
# and hence drop 3000 abortion

In [9]:
# Get all non-birth data, drop 224082 birth
#birth = data[data["Birth Weight"].notnull()]
data = data[data["Birth Weight"].isna()]

In [10]:
# Drop columns: only keep ids instead of diagonosis
droplist = ["Discharge Year", # only 2017
            "Birth Weight", # only NA
            "Abortion Edit Indicator", # after dropping na, only one value, N for no abortion.
            "Operating Certificate Number", # assigned by NYS Department of Health
            "Facility Name", # almost 1-to-1 correspondence to "Permanent Facility Id",
            "CCS Diagnosis Description", # 1-to-1 correspondence to "CCS Diagnosis Code",
            "CCS Procedure Description", # 1-to-1 correspondence to "CCS Procedure Code",
            "APR MDC Description", # 1-to-1 correspondence to "APR MDC Code",
            "APR Severity of Illness Description", # 1-to-1 correspondence to "APR Severity of Illness Code",
            "APR DRG Description",# 1-to-1 correspondence to "APR DRG Code"
            "Payment Typology 1", # Don't care payment type in cost bias
            "Payment Typology 2",
            "Payment Typology 3",
            "Total Costs"]# Total estimated cost for the discharge, corr(charge,cost)~0.7
data = data.drop(droplist, axis=1)
print(np.shape(data))
data.loc[:2]

(2080234, 20)


,Hospital Service Area,Hospital County,Permanent Facility Id,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,Type of Admission,Patient Disposition,CCS Diagnosis Code,CCS Procedure Code,APR DRG Code,APR MDC Code,APR Severity of Illness Code,APR Risk of Mortality,APR Medical Surgical Description,Emergency Department Indicator,Total Charges
1,Hudson Valley,Westchester,1061.0,50 to 69,105,M,White,Spanish/Hispanic,8,Emergency,Skilled Nursing Home,99,108,194,5,3,Moderate,Medical,N,58517.00
2,Hudson Valley,Westchester,1061.0,30 to 49,105,F,White,Unknown,6,Emergency,Court/Law Enforcement,161,102,465,11,2,Moderate,Medical,N,39647.04


In [11]:
data.columns

Index(['Hospital Service Area', 'Hospital County', 'Permanent Facility Id',
       'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race', 'Ethnicity',
       'Length of Stay', 'Type of Admission', 'Patient Disposition',
       'CCS Diagnosis Code', 'CCS Procedure Code', 'APR DRG Code',
       'APR MDC Code', 'APR Severity of Illness Code', 'APR Risk of Mortality',
       'APR Medical Surgical Description', 'Emergency Department Indicator',
       'Total Charges'],
      dtype='object')

In [12]:
data.nunique()

Hospital Service Area                     8
Hospital County                          57
Permanent Facility Id                   210
Age Group                                 5
Zip Code - 3 digits                      50
Gender                                    3
Race                                      4
Ethnicity                                 4
Length of Stay                          120
Type of Admission                         6
Patient Disposition                      20
CCS Diagnosis Code                      258
CCS Procedure Code                      222
APR DRG Code                            284
APR MDC Code                             23
APR Severity of Illness Code              4
APR Risk of Mortality                     4
APR Medical Surgical Description          2
Emergency Department Indicator            2
Total Charges                       1730711
dtype: int64

In [14]:
raw_data.corr(method='pearson', min_periods=1)

,Operating Certificate Number,Permanent Facility Id,Discharge Year,CCS Diagnosis Code,CCS Procedure Code,APR DRG Code,APR MDC Code,APR Severity of Illness Code,Total Charges,Total Costs
Operating Certificate Number,1.000000,0.664918,NaN,0.037160,0.168610,0.042204,0.044093,-0.032443,0.090066,0.065249
Permanent Facility Id,0.664918,1.000000,NaN,0.026997,0.137821,0.053806,0.053181,-0.045941,0.051097,0.027877
Discharge Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCS Diagnosis Code,0.037160,0.026997,NaN,1.000000,0.140015,0.427734,0.454613,-0.195735,-0.088896,-0.058761
CCS Procedure Code,0.168610,0.137821,NaN,0.140015,1.000000,0.221663,0.218536,-0.033895,0.025273,0.021806
APR DRG Code,0.042204,0.053806,NaN,0.427734,0.221663,1.000000,0.952374,-0.115059,-0.090616,-0.063340
APR MDC Code,0.044093,0.053181,NaN,0.454613,0.218536,0.952374,1.000000,-0.118858,-0.068132,-0.044911
APR Severity of Illness Code,-0.032443,-0.045941,NaN,-0.195735,-0.033895,-0.115059,-0.118858,1.000000,0.307326,0.224885
Total Charges,0.090066,0.051097,NaN,-0.088896,0.025273,-0.090616,-0.068132,0.307326,1.000000,0.705678
Total Costs,0.065249,0.027877,NaN,-0.058761,0.021806,-0.063340,-0.044911,0.224885,0.705678,1.000000


In [13]:
data.corr(method='pearson', min_periods=1)

,Permanent Facility Id,CCS Diagnosis Code,CCS Procedure Code,APR DRG Code,APR MDC Code,APR Severity of Illness Code,Total Charges
Permanent Facility Id,1.000000,0.025845,0.131965,0.035231,0.037145,-0.034096,0.062239
CCS Diagnosis Code,0.025845,1.000000,0.142468,0.437407,0.464714,-0.191753,-0.091984
CCS Procedure Code,0.131965,0.142468,1.000000,0.197777,0.199679,0.001375,0.039472
APR DRG Code,0.035231,0.437407,0.197777,1.000000,0.949350,-0.047251,-0.074875
APR MDC Code,0.037145,0.464714,0.199679,0.949350,1.000000,-0.065031,-0.054152
APR Severity of Illness Code,-0.034096,-0.191753,0.001375,-0.047251,-0.065031,1.000000,0.295935
Total Charges,0.062239,-0.091984,0.039472,-0.074875,-0.054152,0.295935,1.000000


In [15]:
data.apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1)

,Hospital Service Area,Hospital County,Permanent Facility Id,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,Type of Admission,Patient Disposition,CCS Diagnosis Code,CCS Procedure Code,APR DRG Code,APR MDC Code,APR Severity of Illness Code,APR Risk of Mortality,APR Medical Surgical Description,Emergency Department Indicator,Total Charges
Hospital Service Area,1.000000,0.448986,0.211807,0.014054,0.700989,0.005842,-0.295032,0.139949,-0.025742,0.042586,-0.025027,-0.003785,0.000091,0.004676,-0.039621,-0.018907,0.008079,0.034122,0.018043,0.027808
Hospital County,0.448986,1.000000,0.418034,0.016121,0.574269,0.004363,-0.297212,0.141414,-0.038562,0.017040,-0.026064,-0.031839,-0.055648,-0.040800,-0.022266,-0.018206,0.020135,-0.032211,0.061492,0.045558
Permanent Facility Id,0.211807,0.418034,1.000000,0.023492,0.250016,-0.010267,-0.156830,0.081173,-0.018041,-0.022665,-0.016602,0.005412,-0.015615,0.006641,0.009078,0.005619,-0.004670,0.006650,0.022913,0.042547
Age Group,0.014054,0.016121,0.023492,1.000000,0.011896,0.054936,-0.037417,0.013194,-0.006627,-0.069892,-0.048294,0.010334,-0.060271,-0.047624,-0.070688,-0.033837,0.085354,-0.100238,0.086535,-0.007568
Zip Code - 3 digits,0.700989,0.574269,0.250016,0.011896,1.000000,0.006807,-0.325856,0.206757,-0.028075,0.089450,-0.021352,-0.002332,0.000153,0.000198,-0.021778,-0.015889,0.008241,0.030012,-0.037396,0.038315
Gender,0.005842,0.004363,-0.010267,0.054936,0.006807,1.000000,0.016624,-0.011132,-0.052156,0.093147,-0.040390,-0.007745,-0.058878,-0.104103,0.030884,0.042482,-0.018048,0.026344,-0.091283,-0.012498
Race,-0.295032,-0.297212,-0.156830,-0.037417,-0.325856,0.016624,1.000000,-0.367445,0.010547,-0.063892,0.015393,-0.003021,-0.012786,-0.015440,0.013159,0.029682,-0.023764,-0.030379,0.023432,-0.008145
Ethnicity,0.139949,0.141414,0.081173,0.013194,0.206757,-0.011132,-0.367445,1.000000,0.012947,0.034789,-0.006550,0.007918,0.019904,0.016248,-0.021451,-0.025989,0.025732,0.021761,-0.025239,0.001784
Length of Stay,-0.025742,-0.038562,-0.018041,-0.006627,-0.028075,-0.052156,0.010547,0.012947,1.000000,-0.012808,0.002109,-0.010959,0.072502,0.052490,0.019111,0.033642,0.172963,0.022607,0.013722,0.036897
Type of Admission,0.042586,0.017040,-0.022665,-0.069892,0.089450,0.093147,-0.063892,0.034789,-0.012808,1.000000,-0.000547,0.091613,0.147798,0.079711,0.091260,0.103193,-0.051599,0.296464,-0.679284,0.011326


From correlation analysis (pd.factorize() transforms categorical to numerical, but could be used in correlation), correlations of different diagonosis are low. There is correlation between geographical info, and also with race.